# Import libraries

In [17]:
import pandas as pd
from pathlib import Path

# Set Variables

In [ ]:
base = Path("D:/kworking/mr_directory/11_grade_control/02_sampling")
yearmonth = "202511"

In [19]:
endorsement = base / f"endorsement/{yearmonth[:4]}/{yearmonth}/cop_delivery_endorsement.csv"
endorsement

WindowsPath('D:/kworking/mr_directory/11_grade_control/02_sampling/endorsement/2025/202511/cop_delivery_endorsement.csv')

In [20]:
results_dir = base / f"results/{yearmonth[:4]}/{yearmonth}"
results_files = [f for f in results_dir.rglob("*.xlsx") if not f.name.startswith("~$")]

In [21]:
logbook = base / f'logbook/{yearmonth[:4]}/{yearmonth}'

# COP Delivery Endorsement

In [22]:
df_e = pd.read_csv(endorsement)

df_e['Lab Number'] = df_e['Unique ID'].astype(str).str.extract(r"(\d+)$").astype(int)
df_e

,Date,Drill Plan,Sample ID,Unique ID,Lab Number
0,11/1/2025,33_LUSONG_225,219,COP25-17716,17716
1,11/1/2025,33_LUSONG_225,204,COP25-17717,17717
2,11/1/2025,NaN,CR-14594,COP25-17718,17718
3,11/1/2025,33_LUSONG_225,217,COP25-17719,17719
4,11/1/2025,33_LUSONG_225,202,COP25-17720,17720
...,...,...,...,...,...
1395,11/27/2025,26_Midlusong_225,2,COP25-19111,19111
1396,11/27/2025,26_Midlusong_225,22,COP25-19112,19112
1397,11/27/2025,NaN,DUP-22,COP25-19113,19113
1398,11/27/2025,26_Midlusong_225,17,COP25-19114,19114


In [23]:
print("Min Lab Number:", df_e["Lab Number"].min())
print("Max Lab Number:", df_e["Lab Number"].max())

Min Lab Number: 17716
Max Lab Number: 19115


## Sequential Validation

In [24]:
lab = df_e["Lab Number"].astype(int)

if not (lab.diff().dropna() == 1).all():
    raise ValueError("Lab Number is NOT sequential.")

# Assay Results

In [25]:
results_data = []

for item in results_files:
    file_name = item.stem
    print(file_name)
    df_raw = pd.read_excel(item)
    start_row = df_raw[
        df_raw.apply(lambda row: row.astype(str).str.contains("Lab").any(), axis=1)
    ].index[0]

    df = pd.read_excel(item, skiprows=start_row)

    end_row = df[
        df.apply(lambda row: row.astype(str).str.contains("TOTAL SAMPLES").any(), axis=1)
    ].index[0]
    df = df.iloc[3:end_row, :8]
    df = df.drop(df.columns[2], axis=1)
    df = df.dropna(how="all")
    df = df.reset_index(drop=True)

    df.columns = [
        "Lab Number",
        "Sample Description",
        "Cu_Orig",
        "Cu_Reassay",
        "CuO_Soluble",
        "Au",
        "Ag",
    ]

    df['file_name'] = file_name

    results_data.append(df)

QAL 2144 COP-0516
QAL 2146 COP-0517
QAL 2147 COP-0518
QAL 2148 COP-0519
QAL 2149 COP-0520
QAL 2155 COP-0521
QAL 2156 COP-0522
QAL 2159 COP-0523
QAL 2161 COP-0524
QAL 2162 COP-0525
QAL 2165 COP-0526
QAL 2166 COP-0527
QAL 2168 COP-0528
QAL 2171 COP-0529
QAL 2171 COP-0530
QAL 2177 COP-0531
QAL 2178 COP-0532
QAL 2180 COP-0533
QAL 2182 COP-0534
QAL 2184 COP-0535
QAL 2186 COP-0536
QAL 2187 COP-0537
QAL 2190 COP-0538
QAL 2199 COP-0539
QAL 2202 COP-0540
QAL 2204 COP-0541
QAL 2208 COP-0542
QAL 2212 COP-0543
QAL 2217 COP-0544
QAL 2223 COP-0545
QAL 2226 COP-0546
QAL 2228 COP-0547
QAL 2230 COP-0548
QAL 2233 COP-0549
QAL 2236 COP-0550
QAL 2252 COP-0551
QAL 2254 COP-0552
QAL 2256 COP-0553
QAL 2257 COP-0554
QAL 2264 COP-0555
QAL 2266 COP-0556
QAL 2269 COP-0557
QAL 2272 COP-0558
QAL 2273 COP-0559


In [26]:
compiled = pd.concat(results_data, ignore_index=True)
compiled["Lab Number"] = compiled["Lab Number"].astype(int)
compiled

,Lab Number,Sample Description,Cu_Orig,Cu_Reassay,CuO_Soluble,Au,Ag,file_name
0,17716,COP-17716,0.06,NaN,0.002,NaN,NaN,QAL 2144 COP-0516
1,17717,COP-17717,0.15,NaN,NaN,NaN,NaN,QAL 2144 COP-0516
2,17718,COP-17718,0.04,NaN,NaN,NaN,NaN,QAL 2144 COP-0516
3,17719,COP-17719,0.08,NaN,0.003,NaN,NaN,QAL 2144 COP-0516
4,17720,COP-17720,0.09,NaN,NaN,NaN,NaN,QAL 2144 COP-0516
...,...,...,...,...,...,...,...,...
1394,19110,COP-19110,0.19,NaN,NaN,NaN,NaN,QAL 2273 COP-0559
1395,19111,COP-19111,0.35,NaN,NaN,NaN,NaN,QAL 2273 COP-0559
1396,19112,COP-19112,0.19,NaN,NaN,NaN,NaN,QAL 2273 COP-0559
1397,19113,COP-19113,0.17,NaN,0.002,NaN,NaN,QAL 2273 COP-0559


In [27]:
print("Min Lab Number:", compiled["Lab Number"].astype(int).min())
print("Max Lab Number:", compiled["Lab Number"].astype(int).max())

Min Lab Number: 17716
Max Lab Number: 19114


## Sequential Validation

In [28]:
lab = compiled["Lab Number"].astype(int)

if not (lab.diff().dropna() == 1).all():
    raise ValueError("Lab Number is NOT sequential.")

# Merge Results

In [29]:
df = pd.merge(df_e, compiled, on="Lab Number")
df

,Date,Drill Plan,Sample ID,Unique ID,Lab Number,Sample Description,Cu_Orig,Cu_Reassay,CuO_Soluble,Au,Ag,file_name
0,11/1/2025,33_LUSONG_225,219,COP25-17716,17716,COP-17716,0.06,NaN,0.002,NaN,NaN,QAL 2144 COP-0516
1,11/1/2025,33_LUSONG_225,204,COP25-17717,17717,COP-17717,0.15,NaN,NaN,NaN,NaN,QAL 2144 COP-0516
2,11/1/2025,NaN,CR-14594,COP25-17718,17718,COP-17718,0.04,NaN,NaN,NaN,NaN,QAL 2144 COP-0516
3,11/1/2025,33_LUSONG_225,217,COP25-17719,17719,COP-17719,0.08,NaN,0.003,NaN,NaN,QAL 2144 COP-0516
4,11/1/2025,33_LUSONG_225,202,COP25-17720,17720,COP-17720,0.09,NaN,NaN,NaN,NaN,QAL 2144 COP-0516
...,...,...,...,...,...,...,...,...,...,...,...,...
1394,11/27/2025,26_Midlusong_225,1,COP25-19110,19110,COP-19110,0.19,NaN,NaN,NaN,NaN,QAL 2273 COP-0559
1395,11/27/2025,26_Midlusong_225,2,COP25-19111,19111,COP-19111,0.35,NaN,NaN,NaN,NaN,QAL 2273 COP-0559
1396,11/27/2025,26_Midlusong_225,22,COP25-19112,19112,COP-19112,0.19,NaN,NaN,NaN,NaN,QAL 2273 COP-0559
1397,11/27/2025,NaN,DUP-22,COP25-19113,19113,COP-19113,0.17,NaN,0.002,NaN,NaN,QAL 2273 COP-0559


In [30]:
print("Min Lab Number:", df["Lab Number"].min())
print("Max Lab Number:", df["Lab Number"].max())

Min Lab Number: 17716
Max Lab Number: 19114


In [31]:
lab = df["Lab Number"].astype(int)

if not (lab.diff().dropna() == 1).all():
    raise ValueError("Lab Number is NOT sequential.")

In [33]:
df.to_csv(f"{logbook}/logbook.csv", index=False)